In [34]:
import MatterSim
import math

def new_simulator(connectivity_dir, scan_data_dir=None):
    # Simulator image parameters
    WIDTH = 640
    HEIGHT = 480
    VFOV = 60

    sim = MatterSim.Simulator()
    if scan_data_dir:
        sim.setDatasetPath(scan_data_dir)
    sim.setNavGraphPath(connectivity_dir)
    sim.setRenderingEnabled(False)
    sim.setCameraResolution(WIDTH, HEIGHT)
    sim.setCameraVFOV(math.radians(VFOV))
    sim.setDiscretizedViewingAngles(True)
    sim.initialize()

    return sim

sim = new_simulator("../../datasets/R2R/connectivity")


In [32]:
scan = "17DRP5sb8fy"
viewpoint = "10c252c90fa24ef3b698c6f54d984c5c"
sim.newEpisode([scan], [viewpoint], [0], [0])

state = sim.getState()[0]

import numpy as np
rgb_bgr = np.array(state.rgb, copy=False)
print("Min:", rgb_bgr.min(), "Max:", rgb_bgr.max(), "Mean:", rgb_bgr.mean())


Min: 0 Max: 0 Mean: 0.0


In [ ]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict, Features, Value, Image
from huggingface_hub import HfApi

api = HfApi()

api.create_repo(repo_id="MP3D_feature", repo_type="dataset", exist_ok=True)

# Step 1: setup
observation_dir = "../../datasets/observations"

features = Features({
    "scan_id": Value("string"),
    "viewpoint_id": Value("string"),
    "image_0": Image(),
    "image_1": Image(),
    "image_2": Image(),
    "image_3": Image()
})

def load_single_scan(scan_id, observation_dir=observation_dir):
    scan_folder = os.path.join(observation_dir, scan_id)
    rows = []
    
    if not os.path.isdir(scan_folder):
        return None  # Skip non-folders

    viewpoint_groups = {}
    for file in os.listdir(scan_folder):
        if file.endswith(".png"):
            try:
                viewpoint_id, direction = file[:-4].split('_')
                direction = int(direction)
            except ValueError:
                print(f"Skipping malformed file name: {file}")
                continue

            key = (scan_id, viewpoint_id)
            if key not in viewpoint_groups:
                viewpoint_groups[key] = [None] * 4

            img_path = os.path.join(scan_folder, file)
            with open(img_path, "rb") as f:
                viewpoint_groups[key][direction] = f.read()

    for (scan_id, viewpoint_id), images in viewpoint_groups.items():
        if all(images):
            rows.append({
                "scan_id": scan_id,
                "viewpoint_id": viewpoint_id,
                "image_0": images[0],
                "image_1": images[1],
                "image_2": images[2],
                "image_3": images[3],
            })

    if rows:
        return pd.DataFrame(rows)
    else:
        return None

/root/miniconda3/envs/vlnhamt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Step 2: build dataset 

# Choose how many scans you want to upload at once (small batches recommended)
scan_ids = os.listdir(observation_dir)
scan_ids.sort()

for i, scan_id in enumerate(scan_ids):
    print(f"Processing scan {i+1}: {scan_id}")

    df_scan = load_single_scan(scan_id, observation_dir)

    dataset = Dataset.from_pandas(df_scan, features=features)

    # Push this dataset as a new split
    dataset.push_to_hub("billzhao1030/MP3D_feature", split=scan_id)
    print(f"Uploaded scan '{scan_id}' as a split.")

In [19]:
from datasets import load_dataset
from tqdm import tqdm
import os
from PIL import Image
from huggingface_hub import hf_hub_download
import json
def load_hub_data(repo_id, filename, extension="json"):
    json_path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        repo_type="dataset"
    )

    if extension.lower() == "json":
        with open(json_path, 'r') as f:
            data = json.load(f)
        return data
    elif extension.lower() == "txt":
        with open(json_path, 'r') as f:
            data = [line.strip() for line in f]
        return data
    else:
        print(f"Unsupported extension: {extension}.")
        return None

# Step 1: setup
observation_dir = "../../datasets/observations"
scan_ids = load_hub_data("billzhao1030/MP3D", "scans.txt", extension="txt")


# Setup
dataset_repo = "billzhao1030/MP3D_feature"
save_dir = "observation"


In [7]:
# Download scan-by-scan
for scan_id in tqdm(scan_ids[:1], desc="Processing scans"):

    # Create the scan folder locally
    scan_folder = os.path.join(save_dir, scan_id)
    os.makedirs(scan_folder, exist_ok=True)

    # If already exists and non-empty, skip
    if os.listdir(scan_folder):
        print(f"Scan {scan_id} already downloaded, skipping.")
        continue

    print(f"\nDownloading split (scan): {scan_id}")
    scan_dataset = load_dataset(dataset_repo, split=scan_id, streaming=True)

    for item in tqdm(scan_dataset, desc=f"Downloading {scan_id}", leave=False):

        viewpoint_id = item["viewpoint_id"]

        # Save four images
        for i in range(4):
            img = item[f"image_{i}"]  # <--- directly get the PIL image!
            img_save_path = os.path.join(scan_folder, f"{viewpoint_id}_{i}.png")
            img.save(img_save_path)

    print(f"✅ Finished scan {scan_id}.\n")

Processing scans:   0%|          | 0/1 [00:00<?, ?it/s]

Processing scans: 100%|██████████| 1/1 [01:08<00:00, 68.89s/it]

✅ Finished scan 17DRP5sb8fy.

